# YOLOモデルの再トレーニング

YOLOモデルを再トレーニングするには、中程度および重度の事故のラベルが付けられた車の画像の準備されたデータセットが必要です。私たちはそのようなデータセット（RoboFlowから取得）を持っており、すでに注釈付きの画像が含まれており、トレーニングセットと検証セットに分割されています。これらのトレーニング/検証セットを使用して現在のYOLOモデルを再トレーニングします。トレーニングに必要なデータ構造に関する情報:

1. モデルに検出させたいオブジェクトのエンコードクラスは、0-`moderate`および1-`severe`です。
2. データセットは自身のフォルダにあり、その中に2つのサブフォルダがあります： `train`と`valid`。各サブフォルダには`images`と`labels`という2つのサブフォルダがあります。
3. 各画像には、`labels`サブフォルダ内に対応する注釈テキストファイルがあります。注釈テキストファイルの名前は画像ファイルと同じです。
4. データセット記述子のYAMLファイル（data.yaml）がデータセットを指し示し、それらに含まれるオブジェクトクラスを記述します。このYAMLファイルはモデルの`train`メソッドに渡され、トレーニングプロセスを開始します。

それでは始めましょう。

In [ ]:
# もしこのLabのために設計されたWorkbenchイメージを使用していない場合、必要なパッケージをインストールするために次の行のコメントを外して実行できます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

import os
import requests
import zipfile
from tqdm.notebook import tqdm
from ultralytics import YOLO

次に、YOLOモデル 'yolo8m.pt' をロードしましょう。

In [ ]:
# モデルをロードします。
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

## トレーニングデータを取得する

以下の2つのトレーニングデータセットを提供しています。これらはzipファイルとして利用可能です：
1) `accident-full.zip`   - モデルを完全に再トレーニングするために使用します。
2) `accident-sample.zip` - モデルを部分的に再トレーニングするために使用します。時間がない場合に利用します。

ワークショップ中はサンプルデータセットのみを使用します。

In [ ]:
# 特定のデータセットを取得するための関数
def retrieve_dataset(dataset_type):

    # ディレクトリが存在するかどうかをチェックし、存在しない場合は作成します
    if not os.path.exists("./datasets/"):
        os.makedirs("./datasets/")

    URL = f"https://rhods-public.s3.amazonaws.com/sample-data/accident-data/accident-{dataset_type}.zip"

    # ファイルが存在するかどうかを確認し、存在しない場合はダウンロードして解凍します
    if not os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Downloading file...")
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(f'./datasets/accident-{dataset_type}.zip', 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    if os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Unzipping file...")
        with zipfile.ZipFile(f'./datasets/accident-{dataset_type}.zip', 'r') as zip_ref:
            zip_ref.extractall(path='./datasets/')
    print("Done!")


dataset_type = 'sample'
# dataset_type = 'full' # フルデータセットを取得したい場合は、この行を使用してください
retrieve_dataset(dataset_type)

## YOLOモデルの再トレーニング

まず、「エポック」が何であるかを理解するところから始めましょう。機械学習モデルは特定のデータセットをアルゴリズムに通じて訓練されます。完全なデータセットがアルゴリズムを通過するたびに、1つの **エポック** が完了したと言います。各 **エポック** はモデルのトレーニングをさらに精緻化します。

以下のトレーニングコードでは、さまざまなパラメータを設定します:  
**results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)**

- epochs: プロセスをデモするために、1エポックしか使用しません。
- imgsz: モデルに供給する画像のサイズです。
- batch: アルゴリズムが同時に処理する画像の数です。結果を改善するにはより多くが望ましいですが、消費するメモリが増えます。ワークショップでリソースが制限されており、これがトレーニングの例であるため、意図的に低い値の2を設定します。

トレーニング実行中に、各 **エポック** はトレーニングフェーズと検証フェーズの両方の要約を示します: 1行目と2行目はトレーニングフェーズの結果を、3行目と4行目は各エポックの検証フェーズの結果を示します。

次のセルを実行して、モデルの再トレーニングを開始しましょう。

In [ ]:
# モデルのトレーニング

results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)

**注意**: 標準的な再トレーニング後、結果に満足している場合、モデルをONNX形式でエクスポートすることができます。以下のコマンドで使用するトレーニングセッションに応じて、trainXを適切なトレーニングセッションに置き換えます。

`ObjDetOXModel = YOLO("runs/detect/trainX/weights/best.pt").export(format="onnx")`

## トレーニング結果の解釈

**フルデータセット**に対するトレーニングのプロセスと結果の詳細な説明は、Labの指示書で入手できます。

モデルを再トレーニングしましたので、これを使用して車の事故が含まれる画像をテストしてみましょう。